In [ ]:
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
)
from dotenv import load_dotenv
import os
from agent import CAMELAgent, get_sys_msgs

In [ ]:

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model_version = "gpt-3.5-turbo" #gpt-4
interviewer = "Joe Roegan"
interviewee = "Sam Altman"

## Init agents

In [ ]:
interviewer_sys_msg, interviewee_sys_msg = get_sys_msgs(
    interviewer, interviewee
)
interviewer_agent = CAMELAgent(interviewer_sys_msg, ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.2, model=model_version))
interviewee_agent = CAMELAgent(interviewee_sys_msg, ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.2, model=model_version))

interviewer_agent.reset()
interviewee_agent.reset()

conversation=[]
interviewee_msg = ""
interviewer_msg = ""

In [ ]:
# director_msg = HumanMessage(
#     content=f"Director instruction : start with a warm welcome !")
# interviewer_msg = interviewer_agent.step(director_msg)

# transcript = f"{interviewer}:\n\n{interviewer_msg.content}\n\n"
# print(transcript)
# conversation.append(transcript)

# interviewee_msg = ""
# interviewer_msg = ""

## Generation loop : run to generate a Q&A step

In [ ]:


step_msg = interviewer_agent.step(interviewee_msg)
interviewer_msg = HumanMessage(content=step_msg.content)
transcript = f"{interviewer}:\n\n{interviewer_msg.content}\n\n"
print(transcript)
conversation.append(transcript)


step_msg = interviewee_agent.step(interviewer_msg)
interviewee_msg = HumanMessage(content=step_msg.content)
transcript = f"{interviewee}:\n\n{interviewee_msg.content}\n\n"
print(transcript)
conversation.append(transcript)

In [ ]:
director_msg = HumanMessage(
    content=f"Director instruction : you're talking too much ! be shorter")
interviewer_msg = interviewer_agent.step(director_msg)

transcript = f"{interviewer}:\n\n{interviewer_msg.content}\n\n"
print(transcript)
conversation.append(transcript)

In [ ]:
interviewer_agent.stored_messages